In [4]:
import torch

torch.__version__

'2.1.0+cu121'

In [1]:
from pathlib import Path

import yaml
import torch
from omegaconf import DictConfig
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks import (
    LearningRateMonitor,
    ModelCheckpoint,
    RichModelSummary,
    RichProgressBar,
)

from src.datamodule.seg import SegDataModule
from src.modelmodule.seg import SegModel


# def main(cfg: dict):  # type: ignore
#     seed_everything(cfg.seed)

#     # init lightning model
#     datamodule = SegDataModule(cfg)
#     # LOGGER.info("Set Up DataModule")
#     model = SegModel(
#         cfg, datamodule.valid_event_df, len(cfg.features), len(cfg.labels), cfg.duration
#     )

    # # set callbacks
    # checkpoint_cb = ModelCheckpoint(
    #     verbose=True,
    #     monitor=cfg.monitor,
    #     mode=cfg.monitor_mode,
    #     save_top_k=1,
    #     save_last=False,
    # )
    # lr_monitor = LearningRateMonitor("epoch")
    # progress_bar = RichProgressBar()
    # model_summary = RichModelSummary(max_depth=2)

    # trainer = Trainer(
    #     # env
    #     default_root_dir=Path.cwd(),
    #     # num_nodes=cfg.training.num_gpus,
    #     accelerator=cfg.accelerator,
    #     precision=16 if cfg.use_amp else 32,
    #     # training
    #     fast_dev_run=cfg.debug,  # run only 1 train batch and 1 val batch
    #     max_epochs=cfg.epoch,
    #     max_steps=cfg.epoch * len(datamodule.train_dataloader()),
    #     gradient_clip_val=cfg.gradient_clip_val,
    #     accumulate_grad_batches=cfg.accumulate_grad_batches,
    #     callbacks=[checkpoint_cb, lr_monitor, progress_bar, model_summary],
    #     logger=None,
    #     # resume_from_checkpoint=resume_from,
    #     num_sanity_val_steps=0,
    #     log_every_n_steps=int(len(datamodule.train_dataloader()) * 0.1),
    #     sync_batchnorm=True,
    #     check_val_every_n_epoch=cfg.check_val_every_n_epoch,
    # )

    # trainer.fit(model, datamodule=datamodule)

    # # load best weights
    # model = model.load_from_checkpoint(
    #     checkpoint_cb.best_model_path,
    #     cfg=cfg,
    #     val_event_df=datamodule.valid_event_df,
    #     feature_dim=len(cfg.features),
    #     num_classes=len(cfg.labels),
    #     duration=cfg.duration,
    # )
    # weights_path = str("model_weights.pth")  # type: ignore

    # torch.save(model.model.state_dict(), weights_path)

    # return

cfg = yaml.safe_load(Path('conf/train.yaml').read_text())
cfg_dir = yaml.safe_load(Path('conf/dir/local.yaml').read_text())
cfg_split = yaml.safe_load(Path('conf/split/fold_0.yaml').read_text())
cfg_model = yaml.safe_load(Path('conf/model/Spec2DCNN.yaml').read_text())
cfg_fe = yaml.safe_load(Path('conf/feature_extractor/CNNSpectrogram.yaml').read_text())
cfg_de = yaml.safe_load(Path('conf/decoder/UNet1DDecoder.yaml').read_text())

cfg['dir'] = cfg_dir
cfg['split'] = cfg_split
cfg['model'] = cfg_model
cfg['feature_extractor'] = cfg_fe
cfg['decoder'] = cfg_de

cfg = DictConfig(cfg)

# main(cfg)

datamodule = SegDataModule(cfg)

x = datamodule.train_dataloader()


In [3]:
num_frames, duration, onset, wakeup, start = 2880, 5760, 280320, 287052, 276195
onset = int((onset - start) / duration * num_frames)
wakeup = int((wakeup - start) / duration * num_frames)
onset, wakeup

(2062, 5428)

In [2]:
labels = None

for i in x:
    labels = i['label']
    break

2880, 5760, 280320, 287052, 276195

2880, 5760, 2062, 5428, 276195


2880, 5760, 349104, 356664, 352705

2880, 5760, -1800, 1979, 352705


2880, 5760, 23040, 28152, 26086

2880, 5760, -1522, 1033, 26086


2880, 5760, 405216, 410652, 404101

2880, 5760, 557, 3275, 404101


2880, 5760, 314460, 321120, 316518

2880, 5760, -1029, 2301, 316518


2880, 5760, 263424, 269856, 258163

2880, 5760, 2630, 5846, 258163


2880, 5760, 108804, 116604, 115259

2880, 5760, -3227, 672, 115259


2880, 5760, 8136, 13092, 8977

2880, 5760, -420, 2057, 8977


2880, 5760, 320304, 327444, 321774

2880, 5760, -735, 2835, 321774


2880, 5760, 8892, 14976, 3645

2880, 5760, 2623, 5665, 3645


2880, 5760, 375744, 376860, 375647

2880, 5760, 48, 606, 375647


2880, 5760, 145608, 152052, 143154

2880, 5760, 1227, 4449, 143154


2880, 5760, 249252, 256860, 248362

2880, 5760, 445, 4249, 248362


2880, 5760, 279132, 285204, 283554

2880, 5760, -2211, 825, 283554


2880, 5760, 286800, 291624, 281708

2880, 5760, 2546, 

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f0638b77370>
Traceback (most recent call last):
  File "/home/alex/Kaggle/DSS/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/home/alex/Kaggle/DSS/.venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1442, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/home/alex/.rye/py/cpython@3.10.11/install/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/home/alex/.rye/py/cpython@3.10.11/install/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/home/alex/.rye/py/cpython@3.10.11/install/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/home/alex/.rye/py/cpython@3.10.11/install/lib/python3.10/selectors.py", lin

In [ ]:
labels.shape

torch.Size([64, 2880, 3])